In [1]:
import pandas as pd


In [10]:
column_name = [ 'variance', 
               'skewness', 
               'curtosis', 
               'entropy',
               'class']
df = pd.read_csv('data_banknote_authentication.csv', header=None, names=column_name)

In [11]:
df.head()

,variance,skewness,curtosis,entropy,class
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


In [22]:
#compute the gini index for a split
def gini_index(splits, classes):
   n_samples = float(sum([len(branch) for branch in splits])) #count the number of samples in total
   
   #gini index for each branch
   gini = 0.0
   for branch in splits:
      size = float(len( branch)) 

      if size == 0:
         continue #avoid divide by 0
      score = 0.0

      for val in classes:
         n = [row[-1] for row in branch].count(val) / size
         score += n * n #score the branch based on each class's score

      gini += (1.0 - score) * (size / n_samples) #calculate the branch score by the size 
   return gini

In [25]:
#test
print(gini_index([[[1, 1], [1, 0]], [[1, 1], [1, 0]]], [0, 1]))
print(gini_index([[[1, 0], [1, 0]], [[1, 1], [1, 1]]], [0, 1]))

group1 = [[2, 0], [4, 0], [6, 0], [8, 1], [10, 1]]
group2 = [[1, 1], [3, 1], [5, 0], [7, 0], [9, 0]]
classes = [0, 1]
print(gini_index([group1, group2], classes))

0.5
0.0
0.48


In [47]:
#split dataset based on a feature and a value before calculating the gini index
def test_split(index, value, dataset):
    left_branch = []
    right_branch = []
    for row in dataset:
        left_branch.append(row) if row[index] < value else right_branch.append(row)
    return left_branch, right_branch
#create left and right branch
#assign to the left if the value of a row on an index is smaller than value

In [86]:
#get the best split based on the gini index calculated for each split 
def get_split(dataset):
  class_values = list(set(row[-1] for row in dataset)) #get all the class values
  best_feature, best_value, best_gini, best_branches = 1000000, 1000000, 1000000, None
  
  #first go through each feature, then go through each row
  #within each row, the value of the feature will be used to calculate split and gini
  for index, feature in enumerate(dataset[0][:-1]):
    for row in dataset:
      branches = test_split(index, row[index], dataset)
      gini = gini_index(branches, class_values)
      print(f'Feature {index} < {row[index]:.3f} Gini={gini:.3f}')
      if gini < best_gini:
        best_feature, best_value, best_gini, best_branches = index, row[index], gini, branches
  return {'feature/index':best_feature, 'value':best_value, 'branches':best_branches}

In [87]:
dataset = [[2.771244718,1.784783929,0],
 [1.728571309,1.169761413,0],
 [3.678319846,2.81281357,0],
 [3.961043357,2.61995032,0],
 [2.999208922,2.209014212,0],
 [7.497545867,3.162953546,1],
 [9.00220326,3.339047188,1],
 [7.444542326,0.476683375,1],
 [10.12493903,3.234550982,1],
 [6.642287351,3.319983761,1]]
split = get_split(dataset)
print(f"The best split: [Feature {split['feature/index']} < {split['value']:.3f}]")

Feature 0 < 2.771 Gini=0.444
Feature 0 < 1.729 Gini=0.500
Feature 0 < 3.678 Gini=0.286
Feature 0 < 3.961 Gini=0.167
Feature 0 < 2.999 Gini=0.375
Feature 0 < 7.498 Gini=0.286
Feature 0 < 9.002 Gini=0.375
Feature 0 < 7.445 Gini=0.167
Feature 0 < 10.125 Gini=0.444
Feature 0 < 6.642 Gini=0.000
Feature 1 < 1.785 Gini=0.500
Feature 1 < 1.170 Gini=0.444
Feature 1 < 2.813 Gini=0.320
Feature 1 < 2.620 Gini=0.417
Feature 1 < 2.209 Gini=0.476
Feature 1 < 3.163 Gini=0.167
Feature 1 < 3.339 Gini=0.444
Feature 1 < 0.477 Gini=0.500
Feature 1 < 3.235 Gini=0.286
Feature 1 < 3.320 Gini=0.375
The best split: [Feature 0 < 6.642]


In [97]:
#decide which class this node belomngs to by checking the most common class among the branch nodes
def to_terminal(branch):
 outcomes = [row[-1] for row in branch]
 return max(set(outcomes), key=outcomes.count)

In [98]:
#create child nodes or terminal node 
def split(node, max_depth, min_row_size, depth):
 left, right = node['branches']
 del(node['branches']) #remove to save memory 

 #check for a no split, if either left or right is empty then create a terminal node
 #no more features to split on and the tree should stop growing
 if not left or not right:
   node['left'] = node['right'] = to_terminal(left + right)
   return 
 
 #check for max depth, if max is reached, make terminal nodes for both left and right branches
 if depth >= max_depth:
   node['left'], node['right'] = to_terminal(left), to_terminal(right)
   return
 
 #process left child
 #if the number of data points in the child node is less than or equal to min_row_size, create a terminal node
 if len(left) <= min_row_size:
   node['left'] = to_terminal(left)
 else:
   node['left'] = get_split(left)
   split(node['left'], max_depth, min_row_size, depth+1)
   
 #process right child
 if len(right) <= min_row_size:
   node['right'] = to_terminal(right)
 else:
   node['right'] = get_split(right)
   split(node['right'], max_depth, min_row_size, depth+1)

In [99]:
# Build a decision tree
def build_tree(train, max_depth, min_size):
 root = get_split(train)
 split(root, max_depth, min_size, 1)
 return root

In [104]:
# Print a decision tree
def print_tree(node, depth=0):
 if isinstance(node, dict):
   print('%s[X%d < %.3f]' % ((depth*' ', (node['feature/index']), node['value'])))
   print_tree(node['left'], depth+1)
   print_tree(node['right'], depth+1)
 else:
   print('%s[%s]' % ((depth*' ', node)))

In [105]:
dataset = [[2.771244718,1.784783929,0],
 [1.728571309,1.169761413,0],
 [3.678319846,2.81281357,0],
 [3.961043357,2.61995032,0],
 [2.999208922,2.209014212,0],
 [7.497545867,3.162953546,1],
 [9.00220326,3.339047188,1],
 [7.444542326,0.476683375,1],
 [10.12493903,3.234550982,1],
 [6.642287351,3.319983761,1]]
tree = build_tree(dataset, 3, 1)
print_tree(tree)

Feature 0 < 2.771 Gini=0.444
Feature 0 < 1.729 Gini=0.500
Feature 0 < 3.678 Gini=0.286
Feature 0 < 3.961 Gini=0.167
Feature 0 < 2.999 Gini=0.375
Feature 0 < 7.498 Gini=0.286
Feature 0 < 9.002 Gini=0.375
Feature 0 < 7.445 Gini=0.167
Feature 0 < 10.125 Gini=0.444
Feature 0 < 6.642 Gini=0.000
Feature 1 < 1.785 Gini=0.500
Feature 1 < 1.170 Gini=0.444
Feature 1 < 2.813 Gini=0.320
Feature 1 < 2.620 Gini=0.417
Feature 1 < 2.209 Gini=0.476
Feature 1 < 3.163 Gini=0.167
Feature 1 < 3.339 Gini=0.444
Feature 1 < 0.477 Gini=0.500
Feature 1 < 3.235 Gini=0.286
Feature 1 < 3.320 Gini=0.375
Feature 0 < 2.771 Gini=0.000
Feature 0 < 1.729 Gini=0.000
Feature 0 < 3.678 Gini=0.000
Feature 0 < 3.961 Gini=0.000
Feature 0 < 2.999 Gini=0.000
Feature 1 < 1.785 Gini=0.000
Feature 1 < 1.170 Gini=0.000
Feature 1 < 2.813 Gini=0.000
Feature 1 < 2.620 Gini=0.000
Feature 1 < 2.209 Gini=0.000
Feature 0 < 2.771 Gini=0.000
Feature 0 < 3.678 Gini=0.000
Feature 0 < 3.961 Gini=0.000
Feature 0 < 2.999 Gini=0.000
Feature 1 < 1

In [106]:
# Make a prediction with a decision tree
def predict(node, row):
 if row[node['index']] < node['value']:
   if isinstance(node['left'], dict):
    return predict(node['left'], row)
   else:
    return node['left']
 else:
   if isinstance(node['right'], dict):
    return predict(node['right'], row)
   else:
    return node['right']

In [107]:
dataset = [[2.771244718,1.784783929,0],
 [1.728571309,1.169761413,0],
 [3.678319846,2.81281357,0],
 [3.961043357,2.61995032,0],
 [2.999208922,2.209014212,0],
 [7.497545867,3.162953546,1],
 [9.00220326,3.339047188,1],
 [7.444542326,0.476683375,1],
 [10.12493903,3.234550982,1],
 [6.642287351,3.319983761,1]]
 
#predict with a stump
stump = {'index': 0, 'right': 1, 'value': 6.642287351, 'left': 0}
for row in dataset:
 prediction = predict(stump, row)
 print('Expected=%d, Got=%d' % (row[-1], prediction))

Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
